In [2]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.api as sm
import yfinance as yf
import requests
from dotenv import load_dotenv

In [30]:
load_dotenv()
api_key = os.getenv("API_KEY")
warnings.filterwarnings('ignore', category=FutureWarning)

# Beta

In [31]:
# Vibra Energia
stock_ticker = "VBBR3.SA"
# S&P 500
market_index_ticker = "^GSPC"
# inicio e fim do período de análise
start_date = "2018-01-01"
end_date = "2022-12-31"

# Download dos dados
stock_data = yf.download(stock_ticker, start=start_date, end=end_date)
market_data = yf.download(market_index_ticker, start=start_date, end=end_date)

# Cálculo de daily returns
stock_returns = stock_data["Adj Close"].pct_change().dropna()
market_returns = market_data["Adj Close"].pct_change().dropna()

# Ajuste dos retornos para o mesmo período
aligned_returns = pd.concat([stock_returns, market_returns], axis=1).dropna()
aligned_returns.columns = ["Stock Returns", "Market Returns"]

# Regressão para o cálculo do Beta
X = sm.add_constant(aligned_returns["Market Returns"])
# Ordinary Least Squares regression (OLS)
model = sm.OLS(aligned_returns["Stock Returns"], X).fit()
beta = model.params["Market Returns"]

# Display the beta
print(f"Beta: {round(beta, 2)}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Beta: 0.74


# Risk free rate

In [32]:
def fetch_treasury_yield(api_key, start_year, end_year):
    """
    Fetches the monthly 10-year treasury yield data from Alpha Vantage API for the specified range of years.

    Parameters:
    - api_key (str): The API key for accessing the Alpha Vantage API.
    - start_year (int): The starting year for fetching the treasury yield data.
    - end_year (int): The ending year for fetching the treasury yield data.

    Returns:
    - treasury_yields (dict): A dictionary containing the treasury yield data for each year in the specified range.
                    The keys are the years and the values are the corresponding treasury yields.
                    If the data for a particular year is not available, the value will be set to None.
    """
    url = "https://www.alphavantage.co/query"
    treasury_yields = {}

    for year in range(start_year, end_year + 1):
        params = {
            "function": "TREASURY_YIELD",
            "interval": "monthly",
            "maturity": "10year",
            "apikey": api_key,
        }

        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            year_data = [
                item for item in data["data"] if item["date"].startswith(str(year))
            ]
            if year_data:
                latest_yield = max(year_data, key=lambda x: x["date"])["value"]
                treasury_yields[year] = float(latest_yield)
            else:
                treasury_yields[year] = None
        else:
            print(f"Could not fetch data for {year}: {response.status_code}")
            treasury_yields[year] = None

    return treasury_yields

In [33]:
# api_key = "demo" <- nunca faça isso
risk_free_rates = fetch_treasury_yield(api_key, 2017, 2022)
risk_free_rates_df = pd.DataFrame(list(risk_free_rates.items()), 
                                  columns=['Year', 'Risk Free Rate'])
display(risk_free_rates_df)

,Year,Risk Free Rate
0,2017,2.40
1,2018,2.83
2,2019,1.86
3,2020,0.93
4,2021,1.47
5,2022,3.62


# Country Risk Premium (CRP)

In [34]:
# Fonte: https://data.worldbank.org/indicator/FR.INR.RISK?locations=BR
crp_data_from_world_bank = "data/API_FR.INR.RISK_DS2_en_csv_v2_6300716_2.csv"


def get_country_risk_premium(country, start_year, end_year):
    """
    Retrieves the country risk premium data for a specific country and time period.

    Parameters:
    - country (str): The name of the country.
    - start_year (int): The starting year of the time period.
    - end_year (int): The ending year of the time period.

    Returns:
    None

    Prints the country risk premium data for the specified country and time period.
    """
    data = pd.read_csv(crp_data_from_world_bank, skiprows=4)
    country_name = data[data["Country Name"] == country]
    year_columns = [str(year) for year in range(start_year, end_year + 1)]
    country_crp_data = country_name[
        ["Country Name", "Country Code", "Indicator Name", "Indicator Code"]
        + year_columns
    ]

    if country == "Brazil":
        country = "Brasil"

    print(
        f"Country Risk Premium para o {country} entre os anos {start_year}–{end_year}:"
    )

    display(country_crp_data)

In [35]:
get_country_risk_premium("Brazil", 2017, 2022)

Country Risk Premium para o Brasil entre os anos 2017–2022:


,Country Name,Country Code,Indicator Name,Indicator Code,2017,2018,2019,2020,2021,2022
29,Brazil,BRA,Risk premium on lending (lending rate minus tr...,FR.INR.RISK,37.859217,31.774033,31.295925,26.2813,22.90315,26.135792


# CAPM

In [47]:
country_risk_premium = 0.0256
market_premium = 0.09
risk_free_rate = risk_free_rates_df.loc[risk_free_rates_df['Year'] == 2022, 'Risk Free Rate'].values[0] / 100

capm = (risk_free_rate + country_risk_premium + (market_premium * beta)) * 100
capm = round(capm, 2)
print(f"CAPM:{capm}%")

CAPM:12.85%


# KD

In [37]:
# Selic - % a.a - Média - Período de 30/12/2022 a 30/12/2022 - Base: Expectativas informadas nos últimos 30 dias
selic_df = pd.read_csv("data/taxa_selic.csv")
tributos = 0.34

def calculate_cdi(selic_df):
    """
    Calculates the CDI (Certificado de Depósito Interbancário) based on the SELIC rate.

    Parameters:
    selic_df (DataFrame): A DataFrame containing the SELIC rate.

    Returns:
    cdi (Series): A Series containing the calculated CDI.

    """
    cdi = selic_df['selic'] / 100
    return cdi

cdi = calculate_cdi(selic_df)

def calculate_kd(cdi):
    """
    Calculates the annualized cost of debt (kd) based on the average CDI rate.

    Parameters:
    cdi (pandas.Series): A pandas Series containing CDI rates.

    Returns:
    float: The annualized cost of debt (kd) rounded to 2 decimal places.
    """
    kd = cdi.mean() * 120
    
    return kd.round(2)

kd = calculate_kd(cdi)

kd_post_tax = (kd * (1 - tributos)).round(2)

print(f"KD: {kd}.")
print(f"KD (pós impostos): {kd_post_tax}.")

KD: 11.16.
KD (pós impostos): 7.37.


# WACC
(CAPM * EQUITY) + (KD(post_tax) * DIVIDA)

In [59]:
de_ratio = 68.1

def calcular_divida(de_ratio):
    de_ratio /= 100
    result = ((de_ratio) / (1 + de_ratio)) * 100
    
    return round(result, 2)

divida = calcular_divida(de_ratio)

equity = 100 - divida

wacc = (capm * equity) + (kd_post_tax * divida)
wacc = round((wacc / 100),2)

print(f"""
CAPM: {capm}%
Equity: {equity}%
kd_post_tax: {kd_post_tax}%
Dívida: {divida}%
WACC: {wacc}%
""")


CAPM: 12.85%
Equity: 59.49%
kd_post_tax: 7.37%
Dívida: 40.51%
WACC: 10.63%

